In [2]:
import numpy as np
import pandas as pd
# df=pd.read_excel("allfeatures.xlsx")
df=pd.read_csv("allfeatures.csv")
print(f'shapte of feateures: {df.shape}')
print(f'total na values: {df.isna().sum().sum()}')
print(f'total null values: {df.isnull().sum().sum()}')

shapte of feateures: (7722, 1389)
total na values: 0
total null values: 0


In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler

X = df.drop(['pcid', 'class', 'smile'], axis=1)  # feature matrix
y = df['class']  # label vector

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature scaling for better performance of KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Setting the parameter grid
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Creating a KNN model
knn = KNeighborsClassifier()

# Using GridSearchCV to find the best parameters
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Evaluating the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# Accuracy
print("Test set accuracy: {:.2f}".format(accuracy_score(y_test, y_pred)))

# Classification Report (includes F1 score, precision, and recall)
print("Classification Report:\n", classification_report(y_test, y_pred))


if len(y.unique()) == 2:
    # Convert to probability scores if y is binary
    y_proba = best_model.predict_proba(X_test_scaled)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    print("ROC AUC Score:", auc)
else:
    print("ROC AUC Score is not computed because the target class is not binary.")

Best parameters: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}
Best cross-validation score: 0.84
Test set accuracy: 0.85
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.88      0.86      1211
           1       0.86      0.81      0.84      1106

    accuracy                           0.85      2317
   macro avg       0.85      0.85      0.85      2317
weighted avg       0.85      0.85      0.85      2317

ROC AUC Score: 0.908139373405029
